# Analysis of the impact of shipping mode on revenue & cost (MLE)

Objective: estimate the impact of `Ship Mode` on revenue (`Sales`) and cost (estimate `Cost = Sales - Profit`) using Maximum Likelihood model (OLS ~ normal distribution).

Using control variables for region (`Region`) and product category (`Category`) to adjust for demand variance.

In [1]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
from pathlib import Path

DATA_PATH = Path("../[DATASET C] Retail Supply Chain Sales Analysis/[C] Retail-Supply-Chain-Sales-Analysis.xlsx")
ORDERS_SHEET = 'Retails Order Full Dataset'

orders = pd.read_excel(DATA_PATH, sheet_name=ORDERS_SHEET)
orders.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Retail Sales People,Product ID,Category,Sub-Category,Product Name,Returned,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Cassandra Brandow,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,Not,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Cassandra Brandow,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Not,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-12-06,2016-12-06,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Anna Andreadi,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,Not,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-11-10,2015-11-10,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Cassandra Brandow,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,Not,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-11-10,2015-11-10,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Cassandra Brandow,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,Not,22.3680,2,0.20,2.5164


### Preprocessing
- Keep rows with valid `Ship Mode`, `Sales`, `Profit`
- Calculate `Cost = Sales - Profit`
- Log1p to reduce skew
- Set columns to categories for model convenience

In [2]:
orders_clean = orders.dropna(subset=['Ship Mode', 'Sales', 'Profit', 'Region', 'Category']).copy()

# Pricing (estimation): Sales - Profit
orders_clean['Cost'] = orders_clean['Sales'] - orders_clean['Profit']

# Normalize column names
orders_clean['ship_mode'] = orders_clean['Ship Mode'].astype('category')
orders_clean['region'] = orders_clean['Region'].astype('category')
orders_clean['category'] = orders_clean['Category'].astype('category')

# Log-transform to stabilize variance
# Use .clip(lower=0) to handle cases where Cost < 0 (because Profit > Sales)
orders_clean['log_sales'] = np.log1p(orders_clean['Sales'])
orders_clean['log_cost'] = np.log1p(orders_clean['Cost'].clip(lower=0))

orders_clean[['ship_mode', 'region', 'category', 'Sales', 'Profit', 'Cost']].head()


,ship_mode,region,category,Sales,Profit,Cost
0,Second Class,South,Furniture,261.9600,41.9136,220.0464
1,Second Class,South,Furniture,731.9400,219.5820,512.3580
2,Second Class,West,Office Supplies,14.6200,6.8714,7.7486
3,Standard Class,South,Furniture,957.5775,-383.0310,1340.6085
4,Standard Class,South,Office Supplies,22.3680,2.5164,19.8516


### MLE model (OLS ~ Gaussian)
Estimation:
- Uncontrolled: `log_sales ~ C(ship_mode)` and `log_cost ~ C(ship_mode)`
- Controlled: `log_sales ~ C(ship_mode) + C(region) + C(category)` and similarly for `log_cost`
`C(...)` automatically creates a dummy variable, defaults to the reference group taken from the category order (usually the first value in alphabetical order).

In [3]:
# Uncontrolled model
model_sales = smf.ols('log_sales ~ C(ship_mode)', data=orders_clean).fit()
model_cost = smf.ols('log_cost ~ C(ship_mode)', data=orders_clean).fit()

# Model with area + category control
model_sales_ctrl = smf.ols('log_sales ~ C(ship_mode) + C(region) + C(category)', data=orders_clean).fit()
model_cost_ctrl = smf.ols('log_cost ~ C(ship_mode) + C(region) + C(category)', data=orders_clean).fit()

print('Revenue (log_sales) - uncontrolled')
print(model_sales.summary().tables[1])
print('Cost (log_cost) - uncontrolled')
print(model_cost.summary().tables[1])

print('=== Revenue (log_sales) - with controlled Region + Category  ===')
print(model_sales_ctrl.summary().tables[1])
print('=== Cost (log cost) - with controlled Region + Category ===')
print(model_cost_ctrl.summary().tables[1])


Revenue (log_sales) - uncontrolled
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          4.1624      0.041    102.676      0.000       4.083       4.242
C(ship_mode)[T.Same Day]           0.0086      0.079      0.108      0.914      -0.147       0.164
C(ship_mode)[T.Second Class]       0.0400      0.054      0.738      0.461      -0.066       0.146
C(ship_mode)[T.Standard Class]    -0.0243      0.045     -0.535      0.593      -0.113       0.065
Cost (log_cost) - uncontrolled
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          3.9616      0.042     95.089      0.000       3.880       4.043
C(ship_mode)[T.Same Day]          -0.0100  

### Expected revenue/cost by shipping method (controlled)
Transform the log coefficients to the original scale (expm1) for comparison. 
Fixed `Region` and `Category` at the most common value to isolate the effect of `Ship Mode`.

In [4]:

# Get modal values ​​for region and category to fix when forecasting
region_mode = orders_clean['region'].mode()[0]
category_mode = orders_clean['category'].mode()[0]

ship_modes = orders_clean['ship_mode'].cat.categories
pred_table = []
for mode in ship_modes:
    df_mode = pd.DataFrame({'ship_mode': [mode], 'region': [region_mode], 'category': [category_mode]})
    pred_log_sales = model_sales_ctrl.predict(df_mode)[0]
    pred_log_cost = model_cost_ctrl.predict(df_mode)[0]
    pred_table.append({
        'Ship Mode': mode,
        'Region (fixed)': region_mode,
        'Category (fixed)': category_mode,
        'Pred Sales (mean)': np.expm1(pred_log_sales),
        'Pred Cost (mean)': np.expm1(pred_log_cost)
    })

pred_df = pd.DataFrame(pred_table).sort_values('Pred Sales (mean)', ascending=False)
pred_df


,Ship Mode,Region (fixed),Category (fixed),Pred Sales (mean),Pred Cost (mean)
2,Second Class,West,Office Supplies,37.803334,26.304284
1,Same Day,West,Office Supplies,36.953449,26.119456
3,Standard Class,West,Office Supplies,36.360069,26.142271
0,First Class,West,Office Supplies,35.942060,26.014697


### Notes
- Positive coefficient in `C(ship_mode)[T.*]` => higher revenue/cost than reference group (standard ship mode) after fixing Region + Category.
  
- `Pred Sales/Pred Cost` provides a practical comparison between ship modes in the same regional & popular category context.
  
- `region_mode`, `category_mode` can be replaced with other values ​​or weighted average to reflect the sales channel reality.